<a href="https://colab.research.google.com/github/skj092/Object-Detection-with-Oxford-IIIT-Pet-Dataset/blob/main/iiit_object_detection_pets_fastai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from fastai.vision.all import *

In [2]:
path = untar_data(URLs.PETS)
Path.BASE_PATH  = path
path.ls()

(#2) [Path('annotations'),Path('images')]

In [3]:
import os
import torch
import torchvision.transforms as T
from torch.utils.data import Dataset
from PIL import Image
import xml.etree.ElementTree as ET

class CustomObjectDetectionDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.xml_files = [file for file in os.listdir(os.path.join(path, 'annotations/xmls')) if file.endswith('.xml')]

    def __len__(self):
        return len(self.xml_files)

    def __getitem__(self, idx):
        xml_file = os.path.join(self.root_dir, 'annotations/xmls', self.xml_files[idx])
        img_name = os.path.splitext(self.xml_files[idx])[0] + '.jpg'
        img_path = os.path.join(self.root_dir, 'images', img_name)

        # Load image
        img = Image.open(img_path).convert("RGB")

        # Load and parse XML annotation
        tree = ET.parse(xml_file)
        root = tree.getroot()

        # Extract image size
        width = int(root.find('size').find('width').text)
        height = int(root.find('size').find('height').text)

        # Initialize lists for target data
        boxes = []
        labels = []

        # Extract bounding box information
        for obj in root.findall('object'):
            label = obj.find('name').text
            xmin = int(obj.find('bndbox').find('xmin').text)
            ymin = int(obj.find('bndbox').find('ymin').text)
            xmax = int(obj.find('bndbox').find('xmax').text)
            ymax = int(obj.find('bndbox').find('ymax').text)

            # Append bounding box coordinates and label
            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(label)

        # Convert boxes and labels to tensors
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.tensor([labels.index(label) for label in labels], dtype=torch.int64)

        # Calculate area (optional)
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

        # Define iscrowd (optional)
        iscrowd = torch.zeros((len(boxes),), dtype=torch.int64)

        # Create target dictionary
        target = {
            "boxes": boxes,
            "labels": labels,
            "image_id": torch.tensor([idx]),
            "area": area,
            "iscrowd": iscrowd
        }

        if self.transform:
            img, target = self.transform(img, target)

        return img, target


In [4]:
import matplotlib.pyplot as plt
import numpy as np

def visualize_dataset_sample(dataset, index, class_names=None):
    """
    Visualizes a single data sample from a dataset.

    Args:
        dataset (Dataset): The PyTorch dataset containing the data samples.
        index (int): The index of the data sample to visualize.
        class_names (list): Optional list of class names corresponding to label indices.
    """
    sample = dataset[index]
    image, target = sample

    image = np.array(image)
    boxes = target["boxes"].numpy()
    labels = target["labels"].numpy()

    plt.figure(figsize=(8, 6))
    plt.imshow(image)
    plt.axis('off')

    for box, label in zip(boxes, labels):
        xmin, ymin, xmax, ymax = box
        rect = plt.Rectangle(
            (xmin, ymin), xmax - xmin, ymax - ymin, fill=False, edgecolor='red', linewidth=2)
        plt.gca().add_patch(rect)

        if class_names:
            label_name = class_names[label]
        else:
            label_name = str(label)

        plt.text(xmin, ymin, label_name, backgroundcolor='red', color='white', fontsize=8)

    plt.show()

In [5]:
# visualize_dataset_sample(ds, 1, class_names=['dog', 'cat'])

In [6]:
from torchvision.transforms import v2 as T


def get_transform(train):
    transforms = []
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    transforms.append(T.ToDtype(torch.float))
    transforms.append(T.ToTensor())
    return T.Compose(transforms)

/usr/local/lib/python3.10/dist-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

In [7]:
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_utils.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_eval.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/transforms.py")

0

In [8]:
import torch
from torch.utils.data import random_split, DataLoader, Subset

ds = CustomObjectDetectionDataset(path, transform=get_transform(train=True))


# Define the dataset size and the desired split ratio
dataset_size = len(ds)
validation_split = 0.2  # 20% of the data will be used for validation

# Calculate the sizes of the training and validation sets
valid_size = int(validation_split * dataset_size)
train_size = dataset_size - valid_size

# Use random_split to split the dataset into train and validation subsets
train_subset, valid_subset = random_split(ds, [train_size, valid_size])

# Create DataLoader objects for train and validation sets
batch_size = 32  # You can adjust this to your preference
train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_subset, batch_size=batch_size, shuffle=False)

# Optionally, you can create a complete dataset for train and validation
train_ds = Subset(ds, train_subset.indices)
valid_ds = Subset(ds, valid_subset.indices)

len(train_ds), len(valid_ds)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `transforms.Compose([transforms.ToImageTensor(), transforms.ConvertImageDtype()])`.
  warnings.warn(


(2949, 737)

In [9]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_dl=DataLoader(train_ds,batch_size=batch_size,shuffle=True,num_workers=2,
                    pin_memory=True if torch.cuda.is_available else False,
                    collate_fn=collate_fn)
val_dl=DataLoader(valid_ds,batch_size=batch_size,shuffle=False,num_workers=2,
                  pin_memory=True if torch.cuda.is_available else False,
                  collate_fn=collate_fn)

len(train_dl), len(val_dl)


(93, 24)

In [10]:
xb, yb = next(iter(train_dl))
xb[0], yb[0]

(tensor([[[0.0824, 0.0863, 0.0902,  ..., 0.0549, 0.0549, 0.0549],
          [0.0824, 0.0863, 0.0902,  ..., 0.0549, 0.0549, 0.0549],
          [0.0824, 0.0863, 0.0902,  ..., 0.0549, 0.0549, 0.0549],
          ...,
          [0.0627, 0.0510, 0.0431,  ..., 0.0039, 0.0039, 0.0039],
          [0.0471, 0.0392, 0.0275,  ..., 0.0039, 0.0039, 0.0039],
          [0.0353, 0.0314, 0.0235,  ..., 0.0039, 0.0039, 0.0039]],
 
         [[0.0549, 0.0588, 0.0627,  ..., 0.0157, 0.0157, 0.0157],
          [0.0549, 0.0588, 0.0627,  ..., 0.0157, 0.0157, 0.0157],
          [0.0549, 0.0588, 0.0627,  ..., 0.0157, 0.0157, 0.0157],
          ...,
          [0.0314, 0.0196, 0.0078,  ..., 0.0039, 0.0039, 0.0039],
          [0.0275, 0.0157, 0.0039,  ..., 0.0039, 0.0039, 0.0039],
          [0.0196, 0.0157, 0.0078,  ..., 0.0039, 0.0039, 0.0039]],
 
         [[0.0314, 0.0353, 0.0392,  ..., 0.0118, 0.0118, 0.0118],
          [0.0314, 0.0353, 0.0392,  ..., 0.0118, 0.0118, 0.0118],
          [0.0314, 0.0353, 0.0392,  ...,

In [11]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# load a model pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 3  # 1 class (person) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [12]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# load a pre-trained model for classification and return
# only the features
backbone = torchvision.models.mobilenet_v2(weights="DEFAULT").features
# ``FasterRCNN`` needs to know the number of
# output channels in a backbone. For mobilenet_v2, it's 1280
# so we need to add it here
backbone.out_channels = 1280

# let's make the RPN generate 5 x 3 anchors per spatial
# location, with 5 different sizes and 3 different aspect
# ratios. We have a Tuple[Tuple[int]] because each feature
# map could potentially have different sizes and
# aspect ratios
anchor_generator = AnchorGenerator(
    sizes=((32, 64, 128, 256, 512),),
    aspect_ratios=((0.5, 1.0, 2.0),)
)

# let's define what are the feature maps that we will
# use to perform the region of interest cropping, as well as
# the size of the crop after rescaling.
# if your backbone returns a Tensor, featmap_names is expected to
# be [0]. More generally, the backbone should return an
# ``OrderedDict[Tensor]``, and in ``featmap_names`` you can choose which
# feature maps to use.
roi_pooler = torchvision.ops.MultiScaleRoIAlign(
    featmap_names=['0'],
    output_size=7,
    sampling_ratio=2,
)

# put the pieces together inside a Faster-RCNN model
model = FasterRCNN(
    backbone,
    num_classes=3,
    rpn_anchor_generator=anchor_generator,
    box_roi_pool=roi_pooler,
)

In [ ]:
# import utils


# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")

# # For Training
# images, targets = next(iter(train_dl))
# images = list(image for image in images)
# targets = [{k: v for k, v in t.items()} for t in targets]
# output = model(images, targets)  # Returns losses and detections
# print(output)

# # For inference
# model.eval()
# x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
# predictions = model(x)  # Returns predictions
# print(predictions[0])

In [13]:
from engine import train_one_epoch, evaluate

# train on the GPU or on the CPU, if a GPU is not available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = 3
# use our dataset and defined transformations
ds = CustomObjectDetectionDataset(path, transform=get_transform(train=True))


# Define the dataset size and the desired split ratio
dataset_size = len(ds)
validation_split = 0.2  # 20% of the data will be used for validation

# Calculate the sizes of the training and validation sets
valid_size = int(validation_split * dataset_size)
train_size = dataset_size - valid_size

# Use random_split to split the dataset into train and validation subsets
train_subset, valid_subset = random_split(ds, [train_size, valid_size])

# Create DataLoader objects for train and validation sets
batch_size = 32  # You can adjust this to your preference
train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_subset, batch_size=batch_size, shuffle=False)

# Optionally, you can create a complete dataset for train and validation
train_ds = Subset(ds, train_subset.indices)
valid_ds = Subset(ds, valid_subset.indices)

train_dl=DataLoader(train_ds,batch_size=batch_size,shuffle=True,num_workers=2,
                    pin_memory=True if torch.cuda.is_available else False,
                    collate_fn=collate_fn)
val_dl=DataLoader(valid_ds,batch_size=batch_size,shuffle=False,num_workers=2,
                  pin_memory=True if torch.cuda.is_available else False,
                  collate_fn=collate_fn)

# get the model using our helper function
# model = get_model_instance_segmentation(num_classes)
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params,
    lr=0.005,
    momentum=0.9,
    weight_decay=0.0005
)

# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=3,
    gamma=0.1
)

# let's train it for 5 epochs
num_epochs = 5

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, train_dl, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, val_dl, device=device)

print("That's it!")

OutOfMemoryError: ignored